<a href="https://colab.research.google.com/github/rishabhranawat/cross-network-patching/blob/main/How_Does_Model_Distillation_Impact_Circuits%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2


Running as a Colab notebook
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.8/174.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00
  Attempting

In [2]:
from functools import partial
from typing import List, Optional, Union, Tuple, Dict

import einops
import numpy as np
import plotly.express as px
import plotly.io as pio
import torch
from circuitsvis.attention import attention_heads
from fancy_einsum import einsum
from IPython.display import HTML, IFrame
from jaxtyping import Float

import transformer_lens.utils as utils
from transformer_lens import ActivationCache, HookedTransformer

import torch

In [3]:
torch.set_grad_enabled(False)
print("Disabled automatic differentiation")

Disabled automatic differentiation


In [4]:
# Load the models
model_gpt2 = HookedTransformer.from_pretrained("gpt2")
model_distil = HookedTransformer.from_pretrained("distilgpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model distilgpt2 into HookedTransformer


In [5]:
import random
import numpy as np
from transformer_lens import HookedTransformer
from typing import List, Tuple

def generate_dataset(size: int = 100) -> Tuple[List[str], List[Tuple[str, str]]]:
    female_names = ["Mary", "Emma", "Olivia", "Ava", "Sophia", "Isabella", "Mia", "Charlotte", "Amelia", "Harper", "Evelyn", "Abigail", "Emily", "Elizabeth", "Sofia", "Avery", "Ella", "Scarlett", "Grace", "Chloe"]
    male_names = ["John", "James", "Robert", "Michael", "William", "David", "Richard", "Joseph", "Thomas", "Charles", "Christopher", "Daniel", "Matthew", "Anthony", "Donald", "Mark", "Paul", "Steven", "Andrew", "Kenneth"]
    occupations = ["friend"]
    adjectives = ["great", "wonderful", "amazing", "brilliant", "talented", "kind", "compassionate", "dedicated", "creative", "passionate", "hardworking", "innovative", "inspiring", "thoughtful", "skilled", "ambitious", "diligent", "reliable", "resourceful", "versatile"]

    def generate_prompt_answer():
        gender = random.choice(["male", "female"])
        name = random.choice(female_names if gender == "female" else male_names)
        occupation = random.choice(occupations)
        adjective = random.choice(adjectives)

        prompt = f"{name} is a {adjective} {occupation}, isn't"
        answer = (" she", " he") if gender == "female" else (" he", " she")

        return prompt, answer

    dataset = [generate_prompt_answer() for _ in range(size)]
    prompts = [item[0] for item in dataset]
    answers = [item[1] for item in dataset]

    return prompts, answers

In [6]:
def logits_to_ave_logit_diff(logits, answer_tokens):
    final_logits = logits[:, -1, :]
    answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
    answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
    return answer_logit_diff.mean()

def calculate_accuracy(predicted_tokens, answer_tokens):
    correct = (predicted_tokens == answer_tokens[:, 0]).sum().item()
    total = len(predicted_tokens)
    return (correct / total) * 100


def get_next_token_prediction(logits, tokens):
    probs = logits.softmax(dim=-1)
    next_token_probs = probs[:, 8, :]  # Use the last input token's position for next token prediction
    top_probs, top_tokens = next_token_probs.topk(1, dim=-1)
    return top_tokens.squeeze(), top_probs.squeeze()

In [7]:
def run_model_with_patching(
    source_model: HookedTransformer,
    target_model: HookedTransformer,
    prompts: List[str],
    answers: List[Tuple[str, str]],
    source_layer_head: Union[int, Tuple[int, int]],
    target_layer_head: Union[int, Tuple[int, int]]
) -> Tuple[float, float, float, float, float, float]:
    """
    Run a model with patching from another model and evaluate performance.

    Args:
    source_model: The model to patch from
    target_model: The model to patch into
    prompts: List of input prompts
    answers: List of tuples containing correct answer pairs (correct, incorrect)
    source_layer_head: Layer number to patch from in the source model, or (layer, head) tuple
    target_layer_head: Layer number to patch into in the target model, or (layer, head) tuple

    Returns:
    Tuple containing:
    - Original model average logit difference
    - Patched model average logit difference
    - Relative change in logit difference
    - Original model accuracy
    - Patched model accuracy
    - Accuracy improvement
    """

    # Tokenize prompts
    tokens = target_model.to_tokens(prompts, prepend_bos=True)

    # Prepare answer tokens
    answer_tokens = torch.tensor([
        [target_model.to_single_token(a[0]), target_model.to_single_token(a[1])]
        for a in answers
    ]).to(target_model.cfg.device)

    def patch_layer_or_head(source_cache, source_layer_head, target_layer_head):
        def hook_fn(act, hook):
            if isinstance(source_layer_head, int):
                # Patch entire layer
                return source_cache[f"blocks.{source_layer_head}.hook_resid_post"]
            else:
                # Patch specific head
                source_layer, source_head = source_layer_head
                target_layer, target_head = target_layer_head
                source_act = source_cache[f"blocks.{source_layer}.hook_attn_out"]
                patched_act = act.clone()
                patched_act[:, target_head, :] = source_act[:, source_head, :]
                return patched_act

        if isinstance(target_layer_head, int):
            hook_name = f"blocks.{target_layer_head}.hook_resid_post"
        else:
            target_layer, target_head = target_layer_head
            hook_name = f"blocks.{target_layer}.hook_attn_out"

        return [(hook_name, hook_fn)]

    # Run the original target model
    original_logits, _ = target_model.run_with_cache(tokens)

    # Run the source model to get its cache
    _, source_cache = source_model.run_with_cache(tokens)

    # Run the target model with patched activations
    patched_logits = target_model.run_with_hooks(
        tokens,
        fwd_hooks=patch_layer_or_head(source_cache, source_layer_head, target_layer_head),
        return_type="logits"
    )

    # Get next token predictions for original and patched models
    original_top_tokens, original_top_probs = get_next_token_prediction(original_logits, tokens)
    patched_top_tokens, patched_top_probs = get_next_token_prediction(patched_logits, tokens)

    # Calculate average logit differences
    original_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
    patched_logit_diff = logits_to_ave_logit_diff(patched_logits, answer_tokens)

    # Calculate relative change
    relative_change = (patched_logit_diff - original_logit_diff) / abs(original_logit_diff) * 100

    # Calculate accuracy for original model
    original_accuracy = calculate_accuracy(original_top_tokens, answer_tokens)

    # Calculate accuracy for patched model
    patched_accuracy = calculate_accuracy(patched_top_tokens, answer_tokens)

    # Calculate improvement
    accuracy_improvement = patched_accuracy - original_accuracy

    return (original_logit_diff.item(), patched_logit_diff.item(), relative_change.item(),
            original_accuracy, patched_accuracy, accuracy_improvement)

In [8]:
def run_experiment(source_model, target_model, source_layer_head, target_layer_head,
                   k_trials: int, dataset_size: int = 100):

    results_list = []

    for _ in range(k_trials):
        prompts, answers = generate_dataset(dataset_size)

        results = run_model_with_patching(
            source_model=source_model,
            target_model=target_model,
            prompts=prompts,
            answers=answers,
            source_layer_head=source_layer_head,
            target_layer_head=target_layer_head
        )

        results_list.append(results)

    # Convert results to numpy array for easy calculation
    results_array = np.array(results_list)

    # Calculate mean and standard deviation for each metric
    means = np.mean(results_array, axis=0)
    stds = np.std(results_array, axis=0)

    # Print results
    metric_names = [
        "Original average logit difference",
        "Patched average logit difference",
        "Relative change",
        "Original Accuracy",
        "Patched Accuracy",
        "Accuracy Improvement"
    ]

    print(f"\nResults over {k_trials} trials:")
    for name, mean, std in zip(metric_names, means, stds):
        print(f"{name}: {mean:.4f} ± {std:.4f}")

# Run the experiment
k_trials = 10  # Number of trials
dataset_size = 100  # Size of each dataset

### (1.) Residual Stream Patching

#### Layer 5 for GPT2-distill vs. Layer 10 GPT2-Small

##### Upswap

In [90]:
run_experiment(
    source_model=model_gpt2,
    target_model=model_distil,
    source_layer_head=10,
    target_layer_head=5,
    k_trials=k_trials,
    dataset_size=dataset_size
)


Results over 10 trials:
Original average logit difference: 0.2325 ± 0.2432
Patched average logit difference: 0.4164 ± 0.3832
Relative change: 121.5969 ± 128.6307
Original Accuracy: 48.0000 ± 2.7203
Patched Accuracy: 40.9000 ± 3.1765
Accuracy Improvement: -7.1000 ± 2.4269


##### Downswap

In [9]:
run_experiment(
    source_model=model_distil,
    target_model=model_gpt2,
    source_layer_head=5,
    target_layer_head=10,
    k_trials=k_trials,
    dataset_size=dataset_size
)


Results over 10 trials:
Original average logit difference: 0.2869 ± 0.2744
Patched average logit difference: 0.1927 ± 0.1240
Relative change: -22.5736 ± 25.2082
Original Accuracy: 48.8000 ± 4.5563
Patched Accuracy: 48.6000 ± 4.7582
Accuracy Improvement: -0.2000 ± 0.8718


## (2.) Specific Head Attention Out Patching



#### L5H8 for distillgpt2 vs. L11H8 for gpt2-small

##### Upswap

In [91]:
run_experiment(
    source_model=model_gpt2,
    target_model=model_distil,
    source_layer_head=(11, 8),
    target_layer_head=(5, 8),
    k_trials=k_trials,
    dataset_size=dataset_size
)


Results over 10 trials:
Original average logit difference: 0.1804 ± 0.2603
Patched average logit difference: 0.1804 ± 0.2603
Relative change: 0.0000 ± 0.0000
Original Accuracy: 49.6000 ± 4.5211
Patched Accuracy: 49.6000 ± 4.5211
Accuracy Improvement: 0.0000 ± 0.0000


##### Downswap

In [92]:
run_experiment(
    source_model=model_distil,
    target_model=model_gpt2,
    source_layer_head=(5, 8),
    target_layer_head=(11, 8),
    k_trials=k_trials,
    dataset_size=dataset_size
)


Results over 10 trials:
Original average logit difference: 0.3654 ± 0.2138
Patched average logit difference: 0.3654 ± 0.2138
Relative change: 0.0000 ± 0.0000
Original Accuracy: 50.9000 ± 5.0882
Patched Accuracy: 51.6000 ± 4.9031
Accuracy Improvement: 0.7000 ± 1.2689


### L5H1 for distillgpt2 vs. L10H9 for gpt2-small


#### Upswap

In [10]:
run_experiment(
    source_model=model_gpt2,
    target_model=model_distil,
    source_layer_head=(10, 9),
    target_layer_head=(5, 1),
    k_trials=k_trials,
    dataset_size=dataset_size
)


Results over 10 trials:
Original average logit difference: 0.2743 ± 0.3564
Patched average logit difference: 0.2743 ± 0.3564
Relative change: 0.0000 ± 0.0000
Original Accuracy: 49.4000 ± 6.0033
Patched Accuracy: 49.4000 ± 6.0033
Accuracy Improvement: 0.0000 ± 0.0000


#### Downswap

In [11]:
run_experiment(
    source_model=model_distil,
    target_model=model_gpt2,
    source_layer_head=(5, 1),
    target_layer_head=(10, 9),
    k_trials=k_trials,
    dataset_size=dataset_size
)


Results over 10 trials:
Original average logit difference: 0.2692 ± 0.2306
Patched average logit difference: 0.2736 ± 0.2031
Relative change: 9.8526 ± 22.0181
Original Accuracy: 49.4000 ± 4.3635
Patched Accuracy: 49.4000 ± 4.3635
Accuracy Improvement: 0.0000 ± 0.0000
